<a href="https://colab.research.google.com/github/tejasnavalkhe/Optimising-Car-Sharing-Profitability-with-a-Regional-Pricing-Strategy/blob/develop/data/Co%20Wheels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Load libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Load data

In [3]:
DATA_PATH = "/content/drive/MyDrive/MSc Dissertation/data/"

In [4]:
bookings_2019_df = pd.read_csv(f"{DATA_PATH}2019 Bookings.csv")
bookings_2020_df = pd.read_csv(f"{DATA_PATH}2020 Bookings.csv")
bookings_2021_df = pd.read_csv(f"{DATA_PATH}2021 Bookings.csv")
bookings_2022_df = pd.read_csv(f"{DATA_PATH}2022 Bookings.csv")
bookings_2023_df = pd.read_csv(f"{DATA_PATH}2023 Bookings.csv")
bookings_2024_df = pd.read_csv(f"{DATA_PATH}2024 Bookings.csv")
accounts_df = pd.read_csv(f"{DATA_PATH}accounts.csv")
users_df = pd.read_csv(f"{DATA_PATH}users.csv")
tariff_df = pd.read_csv(f"{DATA_PATH}Diff Tariffs.csv")


<ipython-input-4-7cab7aa5aa42>:1: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  bookings_2019_df = pd.read_csv(f"{DATA_PATH}2019 Bookings.csv")
<ipython-input-4-7cab7aa5aa42>:3: DtypeWarning: Columns (24,25,26,28,29,30,38) have mixed types. Specify dtype option on import or set low_memory=False.
  bookings_2021_df = pd.read_csv(f"{DATA_PATH}2021 Bookings.csv")
<ipython-input-4-7cab7aa5aa42>:4: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  bookings_2022_df = pd.read_csv(f"{DATA_PATH}2022 Bookings.csv")
<ipython-input-4-7cab7aa5aa42>:5: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  bookings_2023_df = pd.read_csv(f"{DATA_PATH}2023 Bookings.csv")
<ipython-input-4-7cab7aa5aa42>:8: DtypeWarning: Columns (16,30) have mixed types. Specify dtype option on import or set low_memory=False.
  users_df = pd.read_csv(f"{DATA_PATH}us

In [5]:
bookings_df = pd.concat([bookings_2019_df, bookings_2020_df, bookings_2021_df, bookings_2022_df, bookings_2023_df, bookings_2024_df])

In [6]:
bookings_df.head(2)

,account_id,Contract,user_id,vehicle_description,vehicle_registration,vehicle_communication_id,vehicle_operator_name,vehicle_office_use,location_description,location_office_use,...,booking_actual_cost_time,booking_actual_cost_total,booking_credits_used,booking_transactions_value,booking_total_paid,booking_status,booking_ended_early,booking_created_at,booking_cancelled_at,booking_cancellation_reason
0,9ef6f9f2-bb87-11e8-8478-3392d6ed7026,PAYG,5cab890a-bb87-11e8-8c45-8b43f7434530,RETIRED- White Toyota Yaris Hybrid NJ68 VOP Au...,NJ68 VOP,NaN,Co-wheels,PAYG,NaN,NaN,...,5.5,8.38,0,-8.38,8.38,ended,no,01/01/2020 23:30,NaN,NaN
1,837460ea-1a02-11ea-9ddd-919b17a2c51c,PAYG,e4f9362e-1822-11ea-8de1-473e949dd300,RETIRED - White Toyota Aygo X-Play Nav (NL66 NKZ),NL66 NKZ,NaN,Co-wheels,PAYG,Lower Maudlin DNU,NaN,...,9.5,24.26,0,-24.26,24.26,ended,no,01/01/2020 22:56,NaN,NaN


In [7]:
bookings_df.reset_index()

,index,account_id,Contract,user_id,vehicle_description,vehicle_registration,vehicle_communication_id,vehicle_operator_name,vehicle_office_use,location_description,...,booking_actual_cost_time,booking_actual_cost_total,booking_credits_used,booking_transactions_value,booking_total_paid,booking_status,booking_ended_early,booking_created_at,booking_cancelled_at,booking_cancellation_reason
0,0,9ef6f9f2-bb87-11e8-8478-3392d6ed7026,PAYG,5cab890a-bb87-11e8-8c45-8b43f7434530,RETIRED- White Toyota Yaris Hybrid NJ68 VOP Au...,NJ68 VOP,NaN,Co-wheels,PAYG,NaN,...,5.5,8.38,0,-8.38,8.38,ended,no,01/01/2020 23:30,NaN,NaN
1,1,837460ea-1a02-11ea-9ddd-919b17a2c51c,PAYG,e4f9362e-1822-11ea-8de1-473e949dd300,RETIRED - White Toyota Aygo X-Play Nav (NL66 NKZ),NL66 NKZ,NaN,Co-wheels,PAYG,Lower Maudlin DNU,...,9.5,24.26,0,-24.26,24.26,ended,no,01/01/2020 22:56,NaN,NaN
2,2,bcdf83ac-cf78-11e9-a7b5-eba9440aff95,PAYG,5b58ca44-cf78-11e9-8a88-f32ff93f96dc,RETIRED - Renault Zoe EV (DL68 LNK),DL68 LNK,NaN,Co-wheels,PAYG,Bristol - Effingham Road,...,11.0,11.0,0,-11.00,11.00,ended,no,01/01/2020 22:36,NaN,NaN
3,3,10d4ff50-c68a-490b-9831-73a2a53a789e,Franchise,27a79ce4-eeff-4a45-9727-cff211058712,RETIRED - Toyota Yaris Hybrid NAV (NJ18 VTX),NJ18 VTX,NaN,Co-wheels Birmingham,PAYG,NaN,...,38.5,58.66,0,-83.66,0.00,ended,yes,01/01/2020 22:53,NaN,NaN
4,4,ad4720a8-371c-11e8-9c54-45ef183a979b,Contract,84845dc8-a7cb-11e8-a9fa-4fcddbe2d33e,RETIRED - SEAT Ateca 4X4 (NG18 EKY),NG18 EKY,NaN,Co-wheels,Pool Car - Telematics,North Shields - North Tyneside General Hospital,...,0.0,0.0,0,0.00,0.00,ended,no,12/30/2019 23:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721188,53865,c816fa16-114e-11eb-b70a-7de30438bf3e,PAYG,55c15e34-114e-11eb-9853-81bb51f48473,White MG 3 Petrol (Manual) - NG71 ZTF,NG71 ZTF,1929.0,Co-wheels,PAYG,Tunbridge Wells - Goods Station Road,...,47.6,81.04,0,-81.04,81.04,ended,no,12/31/2023 1:13,NaN,NaN
721189,53866,1be16ad0-7528-11ec-a393-15f7512dc1c2,Contract,9a356686-a12c-11ec-b4b4-8f51c3a3a3f1,RETIRED - Black MG 3 (Manual) - Petrol - ND71 KPF,ND71 KPF,1729.0,Co-wheels,Pool Car - Exclusive,Llangefni - Plas Arthur Leisure Centre,...,0.0,0.0,0,0.00,0.00,ended,no,12/30/2023 19:39,NaN,NaN
721190,53867,2093c091-cafa-4136-a474-597a230465ed,PAYG,da59dcb5-5f44-424c-b0a8-2effb8cf9608,White MG 4 Electric Vehicle (Automatic) - FL23...,FL23 HGM,2045.0,Co-wheels,PAYG,Eynsham - Back Lane Car Park,...,23.8,28.12,0,-28.12,0.00,ended,no,8/27/2023 12:48,NaN,NaN
721191,53868,7b9b8e00-5405-11e6-bedd-9b0e013041c9,PAYG,25b18472-5405-11e6-8ab3-c1fe6b6a51e4,White MG 3 Petrol (Manual) - NC71 AAY,NC71 AAY,2287.0,Co-wheels,PAYG - Funded,Glasgow - Kilmarnock Road,...,12.94,14.92,0,-27.02,14.92,ended,no,12/30/2023 16:45,NaN,NaN


In [8]:
bookings_df.head(2)

,account_id,Contract,user_id,vehicle_description,vehicle_registration,vehicle_communication_id,vehicle_operator_name,vehicle_office_use,location_description,location_office_use,...,booking_actual_cost_time,booking_actual_cost_total,booking_credits_used,booking_transactions_value,booking_total_paid,booking_status,booking_ended_early,booking_created_at,booking_cancelled_at,booking_cancellation_reason
0,9ef6f9f2-bb87-11e8-8478-3392d6ed7026,PAYG,5cab890a-bb87-11e8-8c45-8b43f7434530,RETIRED- White Toyota Yaris Hybrid NJ68 VOP Au...,NJ68 VOP,NaN,Co-wheels,PAYG,NaN,NaN,...,5.5,8.38,0,-8.38,8.38,ended,no,01/01/2020 23:30,NaN,NaN
1,837460ea-1a02-11ea-9ddd-919b17a2c51c,PAYG,e4f9362e-1822-11ea-8de1-473e949dd300,RETIRED - White Toyota Aygo X-Play Nav (NL66 NKZ),NL66 NKZ,NaN,Co-wheels,PAYG,Lower Maudlin DNU,NaN,...,9.5,24.26,0,-24.26,24.26,ended,no,01/01/2020 22:56,NaN,NaN


## 2. Data Cleaning

In [9]:
bookings_df.drop(['vehicle_registration', "vehicle_communication_id", "booking_id", "booking_reservation",
               "booking_maintenance", "booking_start", "booking_end", "booking_duration", "booking_rates_overnight",
               "booking_estimated_cost", "booking_credits_used", "booking_transactions_value", "booking_total_paid", "booking_status", "booking_ended_early",
               "booking_created_at", "booking_cancelled_at", "booking_cancellation_reason"], axis=1, inplace=True)

In [10]:
bookings_df.head(2)

,account_id,Contract,user_id,vehicle_description,vehicle_operator_name,vehicle_office_use,location_description,location_office_use,booking_actual_start,booking_actual_end,...,booking_billed_start,booking_billed_end,booking_billed_duration,booking_mileage,booking_tariff,booking_rates_hours,booking_rates_24hours,booking_actual_cost_distance,booking_actual_cost_time,booking_actual_cost_total
0,9ef6f9f2-bb87-11e8-8478-3392d6ed7026,PAYG,5cab890a-bb87-11e8-8c45-8b43f7434530,RETIRED- White Toyota Yaris Hybrid NJ68 VOP Au...,Co-wheels,PAYG,NaN,NaN,01/02/2020 0:11,01/02/2020 0:35,...,01/01/2020 23:45,01/02/2020 0:45,60,16.0,Co-wheels Standard Medium,1.0,0,2.88,5.5,8.38
1,837460ea-1a02-11ea-9ddd-919b17a2c51c,PAYG,e4f9362e-1822-11ea-8de1-473e949dd300,RETIRED - White Toyota Aygo X-Play Nav (NL66 NKZ),Co-wheels,PAYG,Lower Maudlin DNU,NaN,01/01/2020 23:06,01/02/2020 1:24,...,01/01/2020 23:00,01/02/2020 8:00,540,82.0,Co-wheels Standard Small,0.0,0,14.76,9.5,24.26


In [11]:
bookings_filtered_df = bookings_df[bookings_df['Contract'] == 'PAYG']

In [12]:
bookings_filtered_df.head(2)

,account_id,Contract,user_id,vehicle_description,vehicle_operator_name,vehicle_office_use,location_description,location_office_use,booking_actual_start,booking_actual_end,...,booking_billed_start,booking_billed_end,booking_billed_duration,booking_mileage,booking_tariff,booking_rates_hours,booking_rates_24hours,booking_actual_cost_distance,booking_actual_cost_time,booking_actual_cost_total
0,9ef6f9f2-bb87-11e8-8478-3392d6ed7026,PAYG,5cab890a-bb87-11e8-8c45-8b43f7434530,RETIRED- White Toyota Yaris Hybrid NJ68 VOP Au...,Co-wheels,PAYG,NaN,NaN,01/02/2020 0:11,01/02/2020 0:35,...,01/01/2020 23:45,01/02/2020 0:45,60,16.0,Co-wheels Standard Medium,1.0,0,2.88,5.5,8.38
1,837460ea-1a02-11ea-9ddd-919b17a2c51c,PAYG,e4f9362e-1822-11ea-8de1-473e949dd300,RETIRED - White Toyota Aygo X-Play Nav (NL66 NKZ),Co-wheels,PAYG,Lower Maudlin DNU,NaN,01/01/2020 23:06,01/02/2020 1:24,...,01/01/2020 23:00,01/02/2020 8:00,540,82.0,Co-wheels Standard Small,0.0,0,14.76,9.5,24.26


In [13]:
bookings_filtered_df = bookings_filtered_df[(bookings_filtered_df['vehicle_operator_name'] == 'Shropshire') | (bookings_filtered_df['vehicle_operator_name'] == 'Co-wheels')]

In [14]:
bookings_filtered_df.head(2)

,account_id,Contract,user_id,vehicle_description,vehicle_operator_name,vehicle_office_use,location_description,location_office_use,booking_actual_start,booking_actual_end,...,booking_billed_start,booking_billed_end,booking_billed_duration,booking_mileage,booking_tariff,booking_rates_hours,booking_rates_24hours,booking_actual_cost_distance,booking_actual_cost_time,booking_actual_cost_total
0,9ef6f9f2-bb87-11e8-8478-3392d6ed7026,PAYG,5cab890a-bb87-11e8-8c45-8b43f7434530,RETIRED- White Toyota Yaris Hybrid NJ68 VOP Au...,Co-wheels,PAYG,NaN,NaN,01/02/2020 0:11,01/02/2020 0:35,...,01/01/2020 23:45,01/02/2020 0:45,60,16.0,Co-wheels Standard Medium,1.0,0,2.88,5.5,8.38
1,837460ea-1a02-11ea-9ddd-919b17a2c51c,PAYG,e4f9362e-1822-11ea-8de1-473e949dd300,RETIRED - White Toyota Aygo X-Play Nav (NL66 NKZ),Co-wheels,PAYG,Lower Maudlin DNU,NaN,01/01/2020 23:06,01/02/2020 1:24,...,01/01/2020 23:00,01/02/2020 8:00,540,82.0,Co-wheels Standard Small,0.0,0,14.76,9.5,24.26


In [16]:
bookings_filtered_df = bookings_filtered_df.dropna(subset=['location_description', 'location_office_use'], how='all')

In [17]:
bookings_filtered_df.head(2)

,account_id,Contract,user_id,vehicle_description,vehicle_operator_name,vehicle_office_use,location_description,location_office_use,booking_actual_start,booking_actual_end,...,booking_billed_start,booking_billed_end,booking_billed_duration,booking_mileage,booking_tariff,booking_rates_hours,booking_rates_24hours,booking_actual_cost_distance,booking_actual_cost_time,booking_actual_cost_total
1,837460ea-1a02-11ea-9ddd-919b17a2c51c,PAYG,e4f9362e-1822-11ea-8de1-473e949dd300,RETIRED - White Toyota Aygo X-Play Nav (NL66 NKZ),Co-wheels,PAYG,Lower Maudlin DNU,NaN,01/01/2020 23:06,01/02/2020 1:24,...,01/01/2020 23:00,01/02/2020 8:00,540,82.0,Co-wheels Standard Small,0.0,0,14.76,9.5,24.26
2,bcdf83ac-cf78-11e9-a7b5-eba9440aff95,PAYG,5b58ca44-cf78-11e9-8a88-f32ff93f96dc,RETIRED - Renault Zoe EV (DL68 LNK),Co-wheels,PAYG,Bristol - Effingham Road,BRI018,01/01/2020 22:46,01/02/2020 2:01,...,01/01/2020 22:45,01/02/2020 2:30,225,5.0,Co-wheels Standard Electric,0.0,0,0.0,11.0,11.0


In [43]:
tariff_df

,Tariff,Size Category,Vehicle Type,PAYG or Contract,Petrol Or EV,Notes
0,All Saints View,Medium,Everyday,PAYG,EV,NaN
1,Bethesda Family EV,Large,Family,Franchise,EV,NaN
2,"Billing Exempt (24/7) (Default, Co-wheels)",Various,NaN,Contract,Various,NaN
3,Caernarfon Electric,Large,NaN,Franchise,EV,NaN
4,Canterbury Everyday,Medium,Everyday,PAYG,Petrol,NaN
...,...,...,...,...,...,...
84,Dundee City Council Rates (non-electric),Medium,NaN,Contract,Petrol,NaN
85,Hartlepool Standard Small,Small,NaN,Franchise,Petrol,NaN
86,McCarthy & Stone EV,Medium,NaN,Contract,EV,NaN
87,Sunderland City Council Rates,Various,NaN,Contract,Various,NaN


In [ ]:
tariff_df = tariff_df.rename(columns={"Tariff": "booking_tariff"})

In [ ]:
tariff_df.head()

In [ ]:
dff = pd.merge(bookings_filtered_df, tariff_df, how='outer', on='booking_tariff')

In [ ]:
dff.iloc[1]

In [ ]:
dff.head(5)

In [ ]:
# dff.to_excel(f'{DATA_PATH}/combined_file_outer.xlsx')

In [ ]:
dff[dff.isna()]

In [ ]:
dff.isna().sum()

In [ ]:
bookings_filtered.isna().sum()